In [67]:
import pandas as pd
from sklearn.metrics import confusion_matrix

MODEL_NAME = "PTOIE_fine_tuned"


def get_dev_result(model_name: str):
    model_path = "..\evaluations\\" + model_name + "_dev.txt"
    with open(model_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        lines = [line.strip().split(" ") for line in lines]
        y_true = [line[1] for line in lines if len(line) == 3]
        y_pred = [line[2] for line in lines if len(line) == 3]
        return y_true, y_pred
y_true, y_pred = get_dev_result(MODEL_NAME)

def get_confusion_matrix(y_true, y_pred):
    labels = list(set(y_true))
    df = pd.DataFrame(
        data=confusion_matrix(y_true, y_pred, labels=labels),
        columns=labels,
        index=labels,
    )
    df.index.name = "Y-True"
    df.columns.name = "Y-Pred"
    return df

#
# Local (metrics per class)
#
df = get_confusion_matrix(y_true, y_pred)

labels = list(set(y_true))
tps = {}
fps = {}
fns = {}
for label in labels:
    tps[label] = df.loc[label, label]
    fps[label] = df[label].sum() - tps[label]
    fns[label] = df.loc[label].sum() - tps[label]

#
# Global
#
micro_averages = {}
macro_averages = {}

correct_predictions = sum(tps.values())

total_predictions = df.values.sum()
accuracy_global = round(correct_predictions / total_predictions,4 ) if total_predictions > 0. else 0.

print("#-- Local measures --#")
print("True Positives:", tps)
print("False Positives:", fps)
print("False Negatives:", fns)

print("\n#-- Global measures --#")
print("Correct predictions:", correct_predictions)
print("Total predictions:", total_predictions)
print("Accuracy:", accuracy_global)

#-- Local measures --#
True Positives: {'I-ARG0': 62, 'I-V': 13, 'B-V': 37, 'B-ARG0': 42, 'I-ARG1': 27, 'B-ARG1': 16, 'O': 494}
False Positives: {'I-ARG0': 20, 'I-V': 8, 'B-V': 14, 'B-ARG0': 12, 'I-ARG1': 143, 'B-ARG1': 23, 'O': 118}
False Negatives: {'I-ARG0': 22, 'I-V': 56, 'B-V': 14, 'B-ARG0': 9, 'I-ARG1': 29, 'B-ARG1': 35, 'O': 173}

#-- Global measures --#
Correct predictions: 691
Total predictions: 1029
Accuracy: 0.6715


In [68]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

      B-ARG0     0.7778    0.8235    0.8000        51
      B-ARG1     0.4103    0.3137    0.3556        51
         B-V     0.7255    0.7255    0.7255        51
      I-ARG0     0.7561    0.7381    0.7470        84
      I-ARG1     0.1588    0.4821    0.2389        56
         I-V     0.6190    0.1884    0.2889        69
           O     0.8072    0.7406    0.7725       667

    accuracy                         0.6715      1029
   macro avg     0.6078    0.5731    0.5612      1029
weighted avg     0.7299    0.6715    0.6873      1029



In [66]:
df.style.background_gradient(cmap="Purples")

Y-Pred,I-ARG0,I-V,B-V,B-ARG0,I-ARG1,B-ARG1,O
Y-True,,,,,,,
I-ARG0,62,0,1,0,1,1,19
I-V,0,9,3,0,4,7,46
B-V,0,0,36,0,0,0,15
B-ARG0,2,0,0,40,0,0,9
I-ARG1,0,0,0,0,20,0,36
B-ARG1,0,0,0,1,4,15,31
O,14,2,8,8,17,7,611


In [27]:
from transformers import MarianMTModel, MarianTokenizer, pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")
sents = ["The world is very big", "I am a student", "I am a teacher", "I live in Brazil"]
model_name = "Helsinki-NLP/opus-mt-tc-big-en-pt"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
sent = "The Olympic Games take place every four years."
print(pipe(sent))


KeyboardInterrupt

